In [1]:
import time
import os
import copy
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import top_k_accuracy_score
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms


cudnn.benchmark = True
plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def inference(model):
    model.to(device)
    
    start = time.time()
    
    for input, target in dataloaders['val']:
        input, target = input.to(device), target.to(device)
        _ = model(input)
        break
        
    total_time_model = (time.time() - start) / 1024 * 1000

    return total_time_model

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                val_labels = torch.tensor([]).to(device)
                val_outputs = torch.empty((0, len(class_names)), dtype=torch.float32).to(device)

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'val':
                        val_labels = torch.cat((val_labels,  labels))
                        val_outputs = torch.cat((val_outputs,  outputs))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val':
                top1 = top_k_accuracy_score(val_labels.cpu().detach().numpy(), val_outputs.cpu().detach().numpy(), k=1, labels=np.arange(len(class_names)))
                top5 = top_k_accuracy_score(val_labels.cpu().detach().numpy(), val_outputs.cpu().detach().numpy(), k=5, labels=np.arange(len(class_names)))
                print(f'top1: {top1:.4f} top5: {top5:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Чтение данных

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
}


data_dir = '/kaggle/input/russian-road-signs/rtsd-dataset-sm-cls/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

## Обучение модели

In [6]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 240MB/s]


In [7]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=20)

Epoch 0/19
----------


  0%|          | 0/5000 [00:00<?, ?it/s]

train Loss: 1.5378 Acc: 0.5482


  0%|          | 0/1250 [00:00<?, ?it/s]

val Loss: 0.8071 Acc: 0.7482
top1: 0.7482 top5: 0.9562

Epoch 1/19
----------


...

val Loss: 0.5895 Acc: 0.8214
top1: 0.8214 top5: 0.9628

Epoch 19/19
----------


  0%|          | 0/5000 [00:00<?, ?it/s]

train Loss: 0.7387 Acc: 0.7802


  0%|          | 0/1250 [00:00<?, ?it/s]

val Loss: 0.5353 Acc: 0.8328
top1: 0.8328 top5: 0.9712

Training complete in 18m 41s
Best val Acc: 0.840200


In [13]:
print(f"Inference time: {inference(model)} ms")

Inference time: 0.1550882589071989 ms


In [14]:
print(f"Parameters: {count_parameters(model)} ms")

Parameters: 12825 ms
